In [ ]:
import pytraj as pt
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

class TrajectoryBuilder():
    '''
    A wrapper class for calling common trajectory functions
    '''
    def __init__(self,trajectory_file,topology,mask = '@*',skip=1):
        self.computed_dic = {'rmsd' : False, 'k_means' : False}
        self.mask = mask
        self.trajectory_file = trajectory_file
        self.topology = topology
        self.trajectory = pt.Trajectory(trajectory_file, topology)[mask][0::skip]

    def compute_rmsd(self):
        self.rmsd = pt.rmsd(traj=self.trajectory, mask=self.mask)
        self.computed_dic['rmsd'] = True
        return self.rmsd
    
    def plot_rmsd(self):
        if not self.computed_dic['rmsd']:
            self.compute_rmsd()
        df = pd.DataFrame(self.rmsd)
        df.plot()
    
    def compute_kmeans(self,num_clusters):
        self.num_clusters = num_clusters
        self.k_means = pt.cluster.kmeans(self.trajectory,n_clusters=num_clusters)
        self.computed_dic['k_means'] = True
        return self.k_means
    
    def build_MSM(self,num_clusters = None):
        if not self.computed_dic['k_means']:
            self.compute_kmeans(3)
        transition_matrix = [[0] * self.num_clusters for num in range(self.num_clusters)]
        cluster_indices = self.k_means.cluster_index
        for index in range(len(cluster_indices) - 1):
            transition_matrix[cluster_indices[index]][cluster_indices[index+1]] += 1 
        return transition_matrix
    


In [ ]:
def regex_lite(s):
    firstMatch = s.index('[')
    secondMatch = s.index(']')
    f = s[firstMatch+1:secondMatch].split('-')
    arr = []
    bound1 = int(f[0])
    bound2 = int(f[1])
    for i in range(bound1,bound2+1):
        arr.append(f'H{i}')
    return ",".join(arr)

In [ ]:
hydrogens = regex_lite('H[1-10]')
mask = '@C*,H,' + hydrogens

traj = TrajectoryBuilder('Hexane/Hexane_wat_strip.trj','Hexane/Hexane_nowat.prmtop')
trajectory1 = traj.trajectory
trajectory1.top.set_nobox()
trajectory1.superpose(ref=0)
view.add_licorice('@C*')
view = trajectory1.view()
view

In [ ]:
import sys
import math


def build_adjacency(num_atoms,bond_connections):
    '''
    builds adjacency matrix with self loops
    num_atoms (int) : number of atoms in the topology
    bond_connections (array) : connections
    '''
    adjacency_matrix = np.zeros((num_atoms,num_atoms))
    for row in bond_connections:
        adjacency_matrix[row[0]][row[0]] = 1
        adjacency_matrix[row[0]][row[1]] = 1
        adjacency_matrix[row[1]][row[0]] = 1
        adjacency_matrix[row[1]][row[1]] = 1
    return adjacency_matrix


def compute_distance_tensor(trajectory):
    cutoff_vec = np.vectorize(lambda x: 0 if x > 2.5 else 1)
    matrix = pt.analysis.matrix.dist(trajectory)
    return np.vectorize(cutoff_vec)(matrix)
    
    

def compute_degree_matrix(adjacency_matrix):
    shape = adjacency_matrix.shape[0]
    degree_matrix = np.zeros((shape,shape))
    for i,row in enumerate(adjacency_matrix):
        incident_edges = np.sum(row)
        degree_matrix[i][i] = math.sqrt(1/incident_edges)
    
    return degree_matrix



import networkx as nx
matrix = build_adjacency(20,trajectory1.top.bond_indices)

G = nx.Graph(matrix)
pos = nx.kamada_kawai_layout(G)
nx.draw(G,pos)


In [ ]:
matrix2 = compute_distance_tensor(trajectory1)
G2 = nx.Graph(matrix2)
pos = nx.kamada_kawai_layout(G2)
nx.draw(G2,pos)